# Population Simulation Quickstart

This notebook demonstrates how to submit a cohort simulation via the MCP bridge,
poll for completion, and retrieve claim-check artefacts. Configure the
connection details in the first cell before executing the workflow.

## 1. Configure API access

Set `MCP_BASE_URL` and `MCP_TOKEN` in your environment (for example by using
`direnv` or a `.env` file). The confirmation header is required for critical
tools such as `run_population_simulation`.

In [ ]:
import json
import os
import time
from pathlib import Path

import requests

BASE_URL = os.environ.get("MCP_BASE_URL", "http://localhost:8000")
TOKEN = os.environ["MCP_TOKEN"]
HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json",
    "X-MCP-Confirm": "true",
}

def _post(path: str, payload: dict) -> requests.Response:
    url = f"{BASE_URL}{path}"
    response = requests.post(url, headers=HEADERS, data=json.dumps(payload), timeout=30)
    response.raise_for_status()
    return response


## 2. Load a simulation baseline

Provide the path to a `.pkml` model accessible to the bridge. The response
confirms the simulation identifier that subsequent calls must reference.

In [ ]:
model_path = Path("tests/fixtures/demo.pkml").as_posix()
simulation_id = "population-demo"

load_payload = {"filePath": model_path, "simulationId": simulation_id}
load_response = _post("/load_simulation", load_payload)
load_response.json()


## 3. Submit a population simulation job

Here we request a 200-subject Latin Hypercube run with mean and p95 aggregates.
The API immediately returns a job identifier that we can poll.

In [ ]:
population_payload = {
    "modelPath": model_path,
    "simulationId": simulation_id,
    "cohort": {"size": 200, "sampling": "latinHypercube", "seed": 42},
    "outputs": {"aggregates": ["mean", "p95"]},
    "metadata": {"notebook": "population_quickstart"},
}
population_response = _post("/run_population_simulation", population_payload)
job_id = population_response.json()["jobId"]
job_id


## 4. Poll for completion

Poll `get_job_status` until the job leaves the `queued`/`running` states. The
result handle contains a `resultsId` that can be exchanged for aggregates and
chunk metadata.

In [ ]:
status_payload = {"jobId": job_id}
while True:
    status_response = _post("/get_job_status", status_payload)
    job = status_response.json()["job"]
    print(job["status"], job.get("queueWaitSeconds"))
    if job["status"].lower() not in {"queued", "running"}:
        results_handle = job.get("resultHandle", {})
        break
    time.sleep(1.0)

results_handle


## 5. Fetch aggregates and chunk metadata

Population results include both summary aggregates and chunk references.
Streaming large chunk payloads is optional—only download the pieces you need.

In [ ]:
results_id = results_handle["resultsId"]
results_payload = {"resultsId": results_id}
results_response = _post("/get_population_results", results_payload)
results_data = results_response.json()

aggregates = results_data["aggregates"]
first_chunk = results_data["chunks"][0]

aggregates, first_chunk


## 6. (Optional) Download a chunk

Chunks are retrieved via an authenticated `GET` request. Save them to disk for
post-processing in pandas or visualization tooling.

In [ ]:
chunk_id = first_chunk["chunkId"]
chunk_url = f"{BASE_URL}/population_results/{results_id}/chunks/{chunk_id}"
chunk_response = requests.get(chunk_url, headers={"Authorization": f"Bearer {TOKEN}"}, timeout=30)
chunk_response.raise_for_status()

output_path = Path("population_chunk.json")
output_path.write_bytes(chunk_response.content)
output_path
